# What is JuMP?

JuMP is an _modeling language_ for optimization problems, writen in julia. 

When solving an optimization problem, you should use a _solver_: a software implementation of an optimization algorithm. They typically want the problem specified in a much more opaque way, like we saw in the morning.

For larger, more complex problem, writing out the problem explicitly is:
* Time-consuming
* Difficult
* Hard to maintain/extend
* Error-prone

A modeling language (like JuMP) let's you code an optimization problem in a more natural way. It does the translation to the low-level solver format for you.

There are a number of modeling languages out there. Why JuMP?

* User-friendly
* Matches performance of competitors
* Solver-independent
* Easy to extend and take advantage of advanced features


# Installing JuMP

To install JuMP, just run

In [9]:
Pkg.add("JuMP")

INFO: Package JuMP is already installed
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages


We already did this in the preassignment. To actually solve a problem, we will also need to install a solver package. There are 15+ options exposed in julia, each with support for different problem classes, different performance profiles, licensing requirements, etc. For the preassignment, we installed Gurobi, a best-of-breed linear/integer programming solver with a generous academic license.

In [2]:
Pkg.add("Gurobi")

INFO: Package Gurobi is already installed
INFO: METADATA is out-of-date — you may not have the latest version of Gurobi
INFO: Use `Pkg.update()` to get the latest versions of your packages


# A first example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP and Gurobi libraries.

In [3]:
using JuMP, Gurobi

INFO: Recompiling stale cache file /Users/jack/.julia/lib/v0.6/MathProgBase.ji for module MathProgBase.
INFO: Recompiling stale cache file /Users/jack/.julia/lib/v0.6/ReverseDiffSparse.ji for module ReverseDiffSparse.
INFO: Recompiling stale cache file /Users/jack/.julia/lib/v0.6/JuMP.ji for module JuMP.
INFO: Recompiling stale cache file /Users/jack/.julia/lib/v0.6/Gurobi.ji for module Gurobi.


Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.

In [4]:
model = Model(solver=GurobiSolver())

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Gurobi

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro (a fancy function, essentially). The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [5]:
@variable(model, x >= 0)
@variable(model, y >= 0)

y

In [6]:
model

Feasibility problem with:
 * 0 linear constraints
 * 2 variables
Solver is Gurobi

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [8]:
@constraint(model, x + y <= 1)
model

Feasibility problem with:
 * 2 linear constraints
 * 2 variables
Solver is Gurobi

We specify the objective function with the ``@objective`` macro.

In [10]:
@objective(model, Max, x + 2y)

x + 2 y

In [11]:
model

Maximization problem with:
 * 2 linear constraints
 * 2 variables
Solver is Gurobi

To solve the optimization problem, call the ``solve`` function.

In [12]:
solve(model)

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 1 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0040000e+00   4.000000e-03   0.000000e+00      0s
       1    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  2.000000000e+00


:Optimal

We can now inspect the solution values and optimal cost.

In [13]:
getvalue(x)

0.0

In [14]:
getvalue(y)

1.0

In [15]:
getobjectivevalue(model)

2.0

# Exercise

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

In [17]:
model2 = Model(solver=GurobiSolver())
@variable(model2, x >= 0)
@variable(model2, 0<=y<= 1)
@constraint(model2, x + 2y >= 1)
@objective(model2, Min, 3x -y)
solve(model2)

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.000000000e+00


:Optimal